In [1]:
from z3 import *
import math

## Utils

In [2]:
#TODO: sposta gli utils tutti qua su
def flatten(matrix):
    return [e for row in matrix for e in row]

## Encodings

#### At most/least one & exactly one

In [3]:
def at_least_one_seq(bool_vars):
    return Or(bool_vars)

def at_most_one_seq(x, name):
    # bool_vars renamed to x for simplicity
    n = len(x)
    if n == 1:
        return True
    s = [Bool(f"s_{i}_{name}") for i in range(n-1)]     # "name" is used in order to create a
                                                        # unique set of variables s for each constraint
                                                        # of the kind at_most_one_seq, so that they do not overlap 
                                                        # for different constraints!
                                                        # N.B. len(s) = n-1 != n = len (x)
    clauses = []
    clauses.append(Or(Not(x[0]), s[0]))                 # x[0] -> s[0] (s[i] modeled as: s[i] is true if the SUM UP TO index i IS 1!) i.e. x_1 -> s_1 in math notation
    for i in range(1, n-1):
        clauses.append(Or(Not(x[i]), s[i]))             # these two clauses model (x[i] v s[i-1]) -> s[i]
        clauses.append(Or(Not(s[i-1]), s[i]))
        clauses.append(Or(Not(s[i-1]), Not(x[i])))      # this one models s[i-1] -> not x[i]
    clauses.append(Or(Not(s[-1]), Not(x[-1])))          # s[n-2] -> not x[n-1]  i.e. s_(n-1) -> s_n in the mathematical notation (1-based)
    return And(clauses)

def exactly_one_seq(bool_vars, name):
    return And(at_least_one_seq(bool_vars), at_most_one_seq(bool_vars, name))

#### Conversion from int to bin

In [4]:
def num_bits(x):
    return math.floor(math.log2(x))+1

def find_max_digits(s, l): # TODO Idea: optimize number of digits depending on cases
    return num_bits(max(sum(s), max(l)))

In [5]:
def int_to_bin(x, digits):  # TODO read: #https://ericpony.github.io/z3py-tutorial/guide-examples.htm
    x_bin = [(x%(2**(i+1)) // 2**i)==1 for i in range(digits-1,-1,-1)]
    return x_bin

def bin_to_int(x):
    n = len(x)
    x = sum(2**(n-i-1) for i in range(n) if x[i] == 1)
    return x

#### Operations on binary numbers

In [6]:
def equal(v, u, digits):
    return And([v[k]==u[k] for k in range(digits)])

def all_false(v):
    return And([Not(v[k]) for k in range(len(v))])

def leq(v, u, digits):
    """Encoding of v <= u

    Args:
        v (list[bool]): binary representation of v
        u (list[bool]): binary representation of u
        digits (int): number of digits of v and u

    Returns:
        Z3 formula: encoding of v <= u in binary considering their {digits} most significant bits
    """
    assert(len(v) == len(u) and len(u) == digits)
    if digits == 1:
        return Or(v[0]==u[0], And(Not(v[0]), u[0]))
    else:
        return Or(And(Not(v[0]), u[0]), 
                  And(v[0]==u[0], leq(v[1:], u[1:], digits-1)))


In [7]:
def sum_bin(a_bin, b_bin, d_bin, digits, name):
    """Encodes into a SAT formula the binary sum {a_bin + b_bin = d_bin}, each number having {digits} num of bits

    Args:
        a_bin (list[bool]): binary representation of a
        b_bin (list[bool]): binary representation of b
        d_bin (list[bool]): binary representation of d
        digits (int): number of bits of each number 
        name (str): string to identify carry boolean variables

    Returns:
        (Z3-expression): formula representing SAT encoding of binary sum
    """

    #slide SAT-I
    c = [Bool(f"c_{k}_{name}") for k in range(digits+1)]
    c[0] = False
    c[-1] = False

    clauses = []
    for k in range(digits-1,-1,-1):
        # clauses.append((a_bin[k] == b_bin[k]) == (c[k+1] == d_bin[k]))    #TODO: check if it is faster
        clauses.append(d_bin[k] == Or(And(a_bin[k], b_bin[k], c[k+1]), And(a_bin[k], Not(b_bin[k]), Not(c[k+1])),
                                   And(Not(a_bin[k]), b_bin[k], Not(c[k+1])), And(Not(a_bin[k]), Not(b_bin[k]), c[k+1])))
        clauses.append(c[k] == Or(And(a_bin[k],b_bin[k]), And(a_bin[k],c[k+1]), And(b_bin[k],c[k+1])))

    return And(clauses)


def conditional_sum_K_bin(x, alpha, delta, name):
    """Encodes into a SAT formula the constraint {delta = sum_over_j(alpha[j] | x[j] == True)}

    Args: 
        x (list[Bool]): list of Z3 Variables, i.e. x_j tells wether or not to add alpha_j to the sum
        alpha (list[list[bool]]): list of known coefficients, each one represented as list[bool] i.e. binary number, whose subset will be summed in the constraint
        delta (list[Bool]): list of Z3 Variables, which will be constrained to represent the sum
        name (string): to uniquely identify the created variables
    Returns:
        formula (Z3-expression): And of clauses representing SAT encoding of Linear Integer constraint
    
    """
    n = len(x)
    digits = len(delta)

    # matrix containing temporary results of sum_bin
    d = [[Bool(f"d_{j}_{k}_{name}") for k in range(digits)] for j in range(n-1)]    # j = 1..n-1 because last row will be delta
    d.append(delta)

    clauses = []

    # row 0
    clauses.append( And(Implies(x[0], equal(d[0], alpha[0], digits)),              # If x[0] == 1 then d_0 == alpha_0
                        Implies(Not(x[0]), all_false(d[0]))))              # elif x[0] == 0 then d_0 == [0..0]

    # row j>1
    for j in range(1,n):
        clauses.append( And(Implies(x[j], sum_bin(d[j-1], alpha[j], d[j], digits, f"{name}_{j-1}_{j}")),           # If c_j == 1 then d_j == d_j-1 + alpha_j 
                            Implies(Not(x[j]), equal(d[j], d[j-1], digits))))
        
    return And(clauses)

#### Linear Integer constraint over binary numbers

In [8]:
def LinearInteger_bin(x, alpha, beta, name):
    """Encodes into a SAT formula the linear constraint {sum_over_j(alpha[j] | x[j] == True) <= beta}

    Args: 
        x (list[Bool]): list of Z3 Variables, i.e. x_j tells wether or not to add alpha_j to the sum
        alpha (list[list[bool]]): list of known coefficients, each one represented as list[bool] i.e. binary number, whose subset will be summed in the constraint
        beta (list[bool]): binary representing the known term, i.e. the upper bound for the sum
        name (string): to uniquely identify the created variables
    Returns:
        formula (Z3-expression): And of clauses representing SAT encoding of Linear Integer constraint
    """
    n = len(x)
    digits = len(beta)

    clauses = []

    sum_result = [Bool(f"LIsum_result_{k}_{name}") for k in range(digits)]
    clauses.append(conditional_sum_K_bin(x, alpha, sum_result, name))

    # TODO: aggiungere a clause che d_n <= beta
    # I suppose to enter beta as integer (is it the best idea to optimize the program?)
    clauses.append(leq(sum_result, beta, digits)) 
    # TODO: does it make any sense to check d_j <= beta forall j or better to just do it for d_n?
    # TODO: have to check for overflows in num. digits?

    formula = And(clauses)
    return formula



#### Functions to display solution

In [9]:
def interest_variables(model, a):
    return [[j for j in range(len(a[0])) if model.evaluate(a[i][j])] for i in range(len(a))]

def displayMCP(result):
    # TODO: fai piu bello per UNSAT
    if type(result) == str and result=="UNSAT":
        print(result)
    else:
        routes = result[0]
        obj_value = result[1]

#### Functions to display matrices of tensor and dist

In [10]:
def tensor(model, bools):
    m = len(bools)
    n = len(bools[0])
    return [[[model.evaluate(bools[i][j][k]) for k in range(n)] for j in range(n)] for i in range(m)]

def display_matrices(tensor, name):
    m = len(tensor)
    n = len(tensor[0])
    for i in range(m):
        print(f'{name}_{i} matrix:')
        for j in range(n):
            print(tensor[i][j])
        print()

## Assumptions

Assumpions that we can make:
1. $n >= m$
2. The distance matrix $D$ is quasimetric (more info [here](https://proofwiki.org/wiki/Definition:Quasimetric)) => Giving an item to courier who has none is always "same or better" than giving 2 to another courier and none to this one

(Note: 1. + 2. => can add implicit constraint "Every courier has at least one item assigned to it")

## Model 2

#### Objective function and Distance function

In [11]:
def distance_traveled(flat_r_i, flat_D_bin, a_i, distances_i, name):
    """Encodes into a SAT formula the constraint {delta = sum_over_j(alpha[j] | x[j] == True)}

    Args: 
        x (list[Bool]): list of Z3 Variables, i.e. x_j tells wether or not to add alpha_j to the sum
        alpha (list[list[bool]]): list of known coefficients, each one represented as list[bool] i.e. binary number, whose subset will be summed in the constraint
        delta (list[Bool]): list of Z3 Variables, which will be constrained to represent the sum
        name (string): to uniquely identify the created variables
    Returns:
        formula (Z3-expression): And of clauses representing SAT encoding of Linear Integer constraint
    
    """
    n = len(flat_r_i)
    digits = len(flat_D_bin[0][0])

    clause = []

    # matrix containing temporary results of sum_bin
    d = [[Bool(f"d_{j}_{k}_{name}") for k in range(digits)] for j in range(n)]

    # first point
    for k in range(n):
        clause.append( And([Implies(And(a_i[k], flat_r_i[j][k], all_false([flat_r_i[q][x] for q in range(n) for x in range(0, k)])),
                                 equal(d[k], flat_D_bin[n][j], digits))
                            for j in range(n) ]))

    # middle points
    # condizioni della somma: flat_r_i[j][k]=1  flat_r_i[h][p]=1 and all_false(colonne tra k e p)
    # sommo la distanza percorsa fino all'elemento j (della colonna k) + la distanza percorsa tra l'elemento j e p (elemento p della colonna h)
    for k in range(n-1): #k<h
        clause.append( And([Implies(And( a_i[k], a_i[h], flat_r_i[j][k], flat_r_i[p][h], Or(k+1==h, all_false([flat_r_i[q][x] for q in range(n) for x in range(k+1,h)])) ),
                                    sum_bin(d[k], flat_D_bin[j][p], d[h], digits,  f"{name}_{k}_{h}")) 
                            for j in range(n) for p in range(n) for h in range(k+1,n) ]))

    # last point
    for k in range(n):
        clause.append( And([Implies(And(a_i[k], flat_r_i[j][k], all_false([flat_r_i[q][x] for q in range(n) for x in range(k+1, n)])),
                                    sum_bin(d[k], flat_D_bin[j][n], distances_i, digits, f"{name}_last_{k}")) 
                            for j in range(n) ]))

    return And(clause)

def distances_def_constraint_2(distances, r, D_bin, a):
    """Defines the set of binary numbers {distances} as the sum of the binary numbers in {flat_D_bin} whose respective element in {flat_r} is True
        i.e. distances[i] = sum_over_j(flat_D_bin[j] | (flat_r[i])[j] == True)  forall i

    Args:
        distances (list[list[Bool]]): distances[i] will be constrained to be the sum of flat_D_bin[j] with respective flat_r[i][j] == True
        flat_r (list[list[Bool]]]): list of flattened matrices of Z3 Bool variables, each matrix flat_r[i] originally (i.e. before flattening) describing the route of courier i
        flat_D_bin (list[list[bool]]): flattened matrix of distances D, with each element converted to a binary as a list[bool]
    
    Returns:
        (Z3 formula): encoding of the definition of distances
    """
    m = len(distances)

    clauses = []

    for i in range(m):
        clauses.append(distance_traveled(r[i], D_bin, a[i], distances[i], f"distances_def_{i}"))

    return And(clauses)

def obj_function(model, distances):
    m = len(distances)
    k = len(distances[0])
    maxdist = -1
    for i in range(m):
        dist = bin_to_int([1 if model.evaluate(distances[i][j]) else 0 for j in range(k)])
        maxdist = max(maxdist, dist)
    return maxdist

def AllLessEq_bin(distances, obj_value_bin, digits):
    m = len(distances)

    clauses = []

    for i in range(m):
        clauses.append(leq(distances[i], obj_value_bin, digits))

    return And(clauses)


In [12]:
# TODO: add (and copy-implement) timeout decorator
def multiple_couriers_planning_2(m, n, l, s, D):
    """Model 2 in Z3 for the Multiple Couriers Planning problem

    Args:
        m (int): number of couriers
        n (int): number of items to deliver
        l (list[int]): l[i] represents the maximum load of courier i, for i = 1..m
        s (list[int]): s[j] represents the size of item j, for j = 1..n
        D (list[list[int]]): (n+1)x(n+1) matrix, with D[i][j] representing the distance from
                             distribution point i to distribution point j
                    
    """
    ## VARIABLES

    # a for assignments
    a = [[Bool(f"a_{i}_{j}") for j in range(n)] for i in range(m)]
    # a_ij = 1 indicates that courier i takes object j
    # O(m * n) vars

    # r for routes
    r = [[[Bool(f"r_{j}_{k}_{i}") for k in range(n)] for j in range(n)] for i in range(m)]  
    # r_ijk = 1 indicates that courier i deliver to point j as the k-th element in his route
    # n+1 delivery points because considering Origin point as well, representes as n+1-th row and column, for each courier i
    # O(m * n^2) vars

    solver = Solver()

    ## CONSTRAINTS
    # Conversions:
    digits = find_max_digits(s, l)
    s_bin = [int_to_bin(s_j, digits) for s_j in s]
    l_bin = [int_to_bin(l_i, digits) for l_i in l]


    # Constraint 1: every object is assigned to one and only one courier
    for j in range(n):
        solver.add(exactly_one_seq([a[i][j] for i in range(m)], f"assignment_{j}"))
    
    # Constraint 2: every courier can't exceed its load capacity
    for i in range(m):
        solver.add(LinearInteger_bin(a[i], s_bin, l_bin[i], f"load_capacity_courier_{i}"))

    # Constraint 3: every courier has at least 1 item to deliver (implicit constraint, because n >= m and distance is quasimetric (from discussion forum))
    # TODO: check if it actually speeds up execution
    for i in range(m):
        solver.add(at_least_one_seq(a[i]))

    #TODO: better to use "clauses" list than doing so many solver.add below? Or call entirely a function maybe

    # Constraint 4: routes
    for i in range(m):
        # Constraint 4.2: row j has a 1 iff courier i delivers object j
        # rows
        for j in range(n):
            solver.add(Implies(a[i][j], exactly_one_seq(r[i][j], f"courier_{i}_deliver_{j}")))  # If a_ij then exactly_one(r_ij)
            solver.add(Implies(Not(a[i][j]), all_false(r[i][j])))   # else all_false(r_ij)

        # Constraint 4.3: column k has a 1 iff courier i delivers object k
        # columns
        for k in range(n):
            solver.add(Implies(a[i][k], exactly_one_seq([r[i][j][k] for j in range(n)], f"courier_{i}_reaches_{k}")))  # If a_ij then exactly_one(r_i,:,k)
            solver.add(Implies(Not(a[i][k]), all_false([r[i][j][k] for j in range(n)])))   # else all_false(r_i,:,k) 
            # TODO imply false just for the one that weren't alredy counted in the rows (Edo: "Cos?")


    # Optimization search

    # TODO 1: move down
    distance_upper_bound = sum([max(D[i]) for i in range(n+1)])     # TODO: maybe we can find a tighter upper bound on obj func
    dist_digits = num_bits(distance_upper_bound) 

    D_bin = [[int_to_bin(e, dist_digits) for e in D[j]] for j in range(n+1)]

    # distances[i] := binary representation of the distance travelled by courier i
    # TODO 1: move here
    solver.push()   # when backtracking, I want to use less digits! That's why I push the frame now   
    distances = [[Bool(f"dist_bin_{i}_{k}") for k in range(dist_digits)] for i in range(m)]

    # definition of distances using constraints
    solver.add(distances_def_constraint_2(distances, r, D_bin, a)) # define a function in order to call it many times on distances w/ different length! i.e. different number of digits
    
    model = None
    obj_value = None
    while solver.check() == z3.sat:
        model = solver.model()
        obj_value = obj_function(model, distances)    # TODO: check that dist_digits of the modes actually contain the distance travelled by courier (check using r and D)
        print(f"This model obtained objective value: {obj_value}")
        obj_value_digits = num_bits(obj_value - 1)  # <= obj_value - 1  <==>  < obj_value #TODO: could not do this and pass immediately the evaluation of distance with maximum value, smaller overhead...
                                                                                          # BUT then I couldn't print the line above!
        obj_value_bin = int_to_bin(obj_value - 1, obj_value_digits)

        solver.pop()
        solver.push()
        # TODO: IMPORTANT - check if recreating the distances variables every time actually slows down the solver (probably in case of solver that uses explanations/heuristics!)
        # even tho they have less digits every time
        distances = [[Bool(f"dist_bin_{i}_{k}") for k in range(obj_value_digits)] for i in range(m)]
        D_bin =  [[int_to_bin(e, obj_value_digits) for e in D[j]] for j in range(n+1)]       
        solver.add(distances_def_constraint_2(distances, r, D_bin, a)) # TODO: problemi: flat_D_bin[i] e distances non avranno gli stessi bits! distances molti di piu -> easy fix: implementa sum con generico numero di digits (Q: num(a) <= num(b) posso assumerlo??)
        solver.add(AllLessEq_bin(distances, obj_value_bin, obj_value_digits)) # TODO: this time, LInearInteger called inside AllLess must be <, not <= !!! Pass it a parameter to use leq or (to implement) lt (less than)
    
    # TODO: re-do thing above again but for BINARY_SEARCH, not LINEAR_SEARCH, then add parameter to the input of the model to choose between which one to use

    R = tensor(model, r)
    # display_matrices(R, 'r')
    
    if model is None:
        return "UNSAT"

    return (R, obj_value)


#### Test

In [13]:
m = 1
n = 3
l = [15]
s = [3, 2, 4]
D = [[0, 1, 1, 1],
     [1, 0, 1, 1],
     [1, 1, 0, 1],
     [1, 1, 1, 0]]

displayMCP(multiple_couriers_planning_2(m, n, l, s, D))

This model obtained objective value: 4


In [14]:
m = 1
n = 3
l = [15]
s = [3, 2, 4]
D = [[0, 3, 3, 2],
     [3, 0, 4, 3],
     [3, 4, 0, 3],
     [2, 3, 3, 0]]

displayMCP(multiple_couriers_planning_2(m, n, l, s, D))

This model obtained objective value: 12


In [15]:
m = 3
n = 7
l = [15, 10, 7]
s = [3, 2, 6, 8, 5, 4, 4]
D = [[0, 3, 3, 6, 5, 6, 6, 2],
     [3, 0, 4, 3, 4, 7, 7, 3],
     [3, 4, 0, 7, 6, 3, 5, 3],
     [6, 3, 7, 0, 3, 6, 6, 4],
     [5, 4, 6, 3, 0, 3, 3, 3],
     [6, 7, 3, 6, 3, 0, 2, 4],
     [6, 7, 5, 6, 3, 2, 0, 4],
     [2, 3, 3, 4, 3, 4, 4, 0]]

displayMCP(multiple_couriers_planning_2(m, n, l, s, D) )

This model obtained objective value: 18
This model obtained objective value: 12
